In [2]:
import math
import numpy as np
import pandas as pd



In [3]:
df_fin = pd.read_csv('./pickle_files/df_final_spark_top5k_080421.csv',index_col=0)

In [4]:
df_fin_true = df_fin[df_fin['has_enough_stat']]

In [5]:
df_fin_true.head()


,item_code,avg_conv_rate,last_price,last_date,avg_unique_views,PE,PE_err,has_enough_stat,pc2,item_skey
0,1000012530,0.016220,99.99,2021-02-13,66.081387,-0.0187,0.0059,True,44.680,247523697279378
2,1000021922,0.023490,114.99,2021-02-16,90.113095,-0.0194,0.0025,True,NaN,156995084198890
3,1000022179,0.003121,479.99,2021-02-06,93.258739,-0.0136,0.0033,True,276.870,11954970507506
5,1000034743,0.005481,279.99,2021-02-04,110.202494,-0.0186,0.0048,True,171.030,206871238538442
10,1000145006,0.004873,289.99,2021-02-04,188.190476,-0.0086,0.0025,True,188.819,280637889289282


In [6]:
df_final_spark=df_fin_true.astype({'pc2': 'float64'})

print(df_final_spark.dtypes)

df_final_spark.dropna(inplace=True)
df_final_spark = df_final_spark[df_final_spark['PE']< 1.0]
df_final_spark = df_final_spark[df_final_spark['PE_err']<1.0]

item_code             int64
avg_conv_rate       float64
last_price          float64
last_date            object
avg_unique_views    float64
PE                  float64
PE_err              float64
has_enough_stat        bool
pc2                 float64
item_skey             int64
dtype: object


In [7]:
VAT = 1.19




In [8]:
df_n_sales = pd.read_csv('./../raw_data/Excel_files/Top5k_Sales_PE.csv',index_col = 0)

In [9]:
df_n_sales.head()

,Sales_PE,Sales_PE_err
1000021922,-6.5170,1.1057
1000005727,-2.6853,0.8351
1000164660,-9.4314,2.6096
1000176734,-3.0015,0.9799
1000008470,-5.8561,1.6480


In [66]:
df_n_sales['Sales_PE'].mean()

-6.642132061978545

In [67]:
df_n_sales['Sales_PE'].nsmallest(10)



1000123336   -47.5047
1000009278   -34.9123
1000009867   -33.0342
1000125632   -28.8310
1000031306   -23.1860
1000058325   -21.7174
1000183589   -21.1935
1000052582   -20.2912
1000126036   -20.2041
1000215686   -19.6296
Name: Sales_PE, dtype: float64

In [10]:
df_n_sales['item_code'] = df_n_sales.index

In [11]:
df_n_sales.head()

,Sales_PE,Sales_PE_err,item_code
1000021922,-6.5170,1.1057,1000021922
1000005727,-2.6853,0.8351,1000005727
1000164660,-9.4314,2.6096,1000164660
1000176734,-3.0015,0.9799,1000176734
1000008470,-5.8561,1.6480,1000008470


In [12]:
df_merged = pd.merge(df_final_spark,df_n_sales, on="item_code")

In [13]:
df_merged.head()

,item_code,avg_conv_rate,last_price,last_date,avg_unique_views,PE,PE_err,has_enough_stat,pc2,item_skey,Sales_PE,Sales_PE_err
0,1000022179,0.003121,479.99,2021-02-06,93.258739,-0.0136,0.0033,True,276.870,11954970507506,-7.0455,2.1165
1,1000145006,0.004873,289.99,2021-02-04,188.190476,-0.0086,0.0025,True,188.819,280637889289282,-10.1897,2.9552
2,1000005727,0.017079,99.99,2021-02-25,170.099034,-0.0246,0.0044,True,40.729,71488185050378,-2.6853,0.8351
3,1000016028,0.016290,139.99,2021-02-25,26.446154,-0.0328,0.0057,True,52.430,48800168481821,-4.4643,0.8350
4,1000175476,0.055748,83.99,2021-02-25,80.722222,-0.0505,0.0061,True,48.799,115260611693891,-2.5839,0.4655


In [14]:
df_merged['avg_sales'] = df_merged['avg_unique_views']*df_merged['avg_conv_rate']


In [15]:
def create_auxiliary_var(dataf=df_merged):
    dataf['Delta_price'] = (dataf['last_price']/VAT - dataf['pc2'])
    dataf['Margin_base'] = dataf['avg_sales']*(dataf['Delta_price'])

def classify(dataf= df_merged):
    dataf['Group_flag'] = np.where(dataf['Margin_base'] < (dataf['Margin_up'] - 3*dataf['Margin_err_up']), 'up',
                                        np.where(dataf['Margin_base'] < (dataf['Margin_down'] - 3*dataf['Margin_err_down']), 'down','neutral'))

def classify_up(dataf=df_merged):
    dataf['Group_flag'] = np.where(dataf['Margin_base'] < (dataf['Margin_up'] - 3*dataf['Margin_err_up']), 'up','neutral_up')

def classify_down(dataf=df_merged):
    dataf['Group_flag'] = np.where(dataf['Margin_base'] < (dataf['Margin_down'] - 3*dataf['Margin_err_down']), 'down','neutral_down')

In [16]:
def create_Margin( increment = 0.01, dataf = df_merged, flag = 'up'):
    if flag == 'down':
        increment = -increment

    price          = 'price_'+flag
    Delta_price    = 'Delta_price_'+flag
    sales          = 'avg_sales_'+flag
    sales_err      = 'avg_sales_err_'+flag
    Margin         = 'Margin_'+flag
    Margin_err     = 'Margin_err_'+flag

    #CROSS CHECK THE WHOLE ERROR PROPAGATION!!!!

    dataf[price]              =  dataf['last_price']*math.exp(increment)
    dataf[Delta_price]        =  (dataf[price]/VAT - dataf['pc2'])
    dataf[sales]              =  dataf['avg_sales']*np.exp(dataf['Sales_PE']*np.log(dataf[price]/dataf['last_price']))
    dataf[sales_err]          =  np.fabs(dataf[sales])*np.fabs(dataf['Sales_PE_err']*np.log(dataf[price]/dataf['last_price']))
    dataf[Margin]             =  dataf[sales]*(dataf[Delta_price])
    #Propagate Margin error
    dataf[Margin_err]         =  dataf[sales_err]*dataf[Delta_price]



In [17]:
#print(df_final_spark[['item_code','avg_conv_rate','last_price']].head())
#create_Margin(0.01)

In [46]:
original_df= df_merged
print(len(original_df.index))
tmp_df = pd.DataFrame()

original_df_up   = pd.DataFrame()
original_df_down = pd.DataFrame()

710


In [47]:
list_of_columns_up   = ['item_code','avg_conv_rate','avg_unique_views','last_date','last_price','price_up','PE','PE_err','Sales_PE','Sales_PE_err','Delta_price','pc2','avg_sales','avg_sales_up','avg_sales_err_up','Margin_base','Margin_up','Group_flag']
list_of_columns_down = ['item_code','avg_conv_rate','avg_unique_views','last_date','last_price','price_down','PE','PE_err','Sales_PE','Sales_PE_err','Delta_price','pc2','avg_sales','avg_sales_down','avg_sales_err_down','Margin_base','Margin_down','Group_flag']
create_auxiliary_var(original_df)
for inc in [0.01,0.02,0.03,0.04,0.05,0.1,0.15,0.2,0.25,0.3]:
    print('Increment = ' + str(inc))
    if inc == 0.01:
        create_Margin(inc,original_df,'up')
        create_Margin(inc,original_df,'down')
        classify(original_df)
        tmp_df           = tmp_df.append(original_df[original_df['Group_flag'] == 'neutral'])
        original_df_up   = original_df_up.append(original_df[original_df['Group_flag'] == 'up'])
        original_df_up   = original_df_up[list_of_columns_up]
        original_df_down = original_df_down.append(original_df[original_df['Group_flag'] == 'down'])
        original_df_down   = original_df_down[list_of_columns_down]
        #print('tmp\n',tmp_df[list_of_columns].head())
        #print('up\n', original_df_up[list_of_columns].head())
        #print('down\n',original_df_down[list_of_columns].head())

    if inc > 0.01:
        create_Margin(inc,original_df_up,'up')
        create_Margin(inc,original_df_down,'down')
        classify_up(original_df_up)
        classify_down(original_df_down)
        tmp_df           = tmp_df.append(original_df_up[original_df_up['Group_flag'] == 'neutral_up'])
        tmp_df           = tmp_df.append(original_df_down[original_df_down['Group_flag'] == 'neutral_down'])
        original_df_up   = original_df_up[original_df_up['Group_flag'] == 'up']
        original_df_down = original_df_down[original_df_down['Group_flag'] == 'down']
        #print('tmp\n',tmp_df[list_of_columns].head())
        #print('up\n', original_df_up[list_of_columns].head())
        #print('down\n',original_df_down[list_of_columns].head())
    original_df_down['Margin_base']   = original_df_down['Margin_down']
    original_df_up['Margin_base']   = original_df_up['Margin_up']

    print(len(tmp_df.index))
    print(len(original_df_up.index))
    print(len(original_df_down.index))

Increment = 0.01
589
34
87
Increment = 0.02
696
14
0
Increment = 0.03
700
10
0
Increment = 0.04
703
7
0
Increment = 0.05
705
5
0
Increment = 0.1
705
5
0
Increment = 0.15
705
5
0
Increment = 0.2
706
4
0
Increment = 0.25
706
4
0
Increment = 0.3
706
4
0


In [48]:
tmp_df
#print(or_filt_df[or_filt_df['Group_flag'] =='down'])
#print(or_filt_df[or_filt_df['Group_flag'] =='up'])
#print(or_filt_df[or_filt_df['Group_flag'] =='neutral'])

,item_code,avg_conv_rate,last_price,last_date,avg_unique_views,PE,PE_err,has_enough_stat,pc2,item_skey,...,avg_sales_err_up,Margin_up,Margin_err_up,price_down,Delta_price_down,avg_sales_down,avg_sales_err_down,Margin_down,Margin_err_down,Group_flag
0,1000022179,0.003121,479.99,2021-02-06,93.258739,-0.0136,0.0033,True,276.870,1.195497e+13,...,0.005742,35.413232,0.749521,475.214020,122.469512,0.312341,0.006611,38.252282,0.809610,neutral
1,1000145006,0.004873,289.99,2021-02-04,188.190476,-0.0086,0.0025,True,188.819,2.806379e+14,...,0.024478,47.476716,1.403032,287.104551,52.445329,1.015517,0.030011,53.259126,1.573914,neutral
2,1000005727,0.017079,99.99,2021-02-25,170.099034,-0.0246,0.0044,True,40.729,7.148819e+13,...,0.023618,124.836973,1.042514,98.995083,42.460145,2.984204,0.024921,126.709720,1.058153,neutral
4,1000175476,0.055748,83.99,2021-02-25,80.722222,-0.0505,0.0061,True,48.799,1.152606e+14,...,0.020414,98.625979,0.459104,83.154286,21.078551,4.617874,0.021496,97.338083,0.453109,neutral
5,1000176734,0.007736,159.99,2021-02-08,100.555556,-0.0149,0.0030,True,54.119,3.825599e+13,...,0.007397,61.656543,0.604172,158.398073,78.988624,0.801580,0.007855,63.315723,0.620431,neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
411,1000196335,0.000697,549.99,2021-01-06,112.662565,0.0001,0.0000,NaN,334.639,NaN,...,0.000644,11.076589,0.094240,NaN,NaN,NaN,NaN,NaN,NaN,neutral_up
651,1000041395,0.215808,16.99,2021-02-25,34.981203,-0.1873,0.0388,NaN,11.710,NaN,...,0.102481,22.250684,0.322813,NaN,NaN,NaN,NaN,NaN,NaN,neutral_up
520,1000184398,0.079964,57.99,2021-02-07,21.415294,-0.0610,0.0115,NaN,43.549,NaN,...,0.024366,11.863193,0.187142,NaN,NaN,NaN,NaN,NaN,NaN,neutral_up
654,1000002470,0.068160,54.99,2021-02-25,30.527679,-0.0478,0.0071,NaN,39.250,NaN,...,0.026256,17.489979,0.244947,NaN,NaN,NaN,NaN,NaN,NaN,neutral_up


In [49]:
tmp_df.set_index(['Group_flag','item_code']).count(level='Group_flag')
#554 neutral, 4 down ,440 up

,avg_conv_rate,last_price,last_date,avg_unique_views,PE,PE_err,has_enough_stat,pc2,item_skey,Sales_PE,...,avg_sales_up,avg_sales_err_up,Margin_up,Margin_err_up,price_down,Delta_price_down,avg_sales_down,avg_sales_err_down,Margin_down,Margin_err_down
Group_flag,,,,,,,,,,,,,,,,,,,,,
neutral,589,589,589,589,589,589,589,589,589,589,...,589,589,589,589,589,589,589,589,589,589
neutral_down,87,87,87,87,87,87,0,87,0,87,...,0,0,0,0,87,87,87,87,87,87
neutral_up,30,30,30,30,30,30,0,30,0,30,...,30,30,30,30,0,0,0,0,0,0


In [50]:
tmp_df['Margin_base'] = df_merged['Margin_base']
tmp_df.head()
#df_final_spark.head(400)


,item_code,avg_conv_rate,last_price,last_date,avg_unique_views,PE,PE_err,has_enough_stat,pc2,item_skey,...,avg_sales_err_up,Margin_up,Margin_err_up,price_down,Delta_price_down,avg_sales_down,avg_sales_err_down,Margin_down,Margin_err_down,Group_flag
0,1000022179,0.003121,479.99,2021-02-06,93.258739,-0.0136,0.0033,True,276.870,1.195497e+13,...,0.005742,35.413232,0.749521,475.214020,122.469512,0.312341,0.006611,38.252282,0.809610,neutral
1,1000145006,0.004873,289.99,2021-02-04,188.190476,-0.0086,0.0025,True,188.819,2.806379e+14,...,0.024478,47.476716,1.403032,287.104551,52.445329,1.015517,0.030011,53.259126,1.573914,neutral
2,1000005727,0.017079,99.99,2021-02-25,170.099034,-0.0246,0.0044,True,40.729,7.148819e+13,...,0.023618,124.836973,1.042514,98.995083,42.460145,2.984204,0.024921,126.709720,1.058153,neutral
4,1000175476,0.055748,83.99,2021-02-25,80.722222,-0.0505,0.0061,True,48.799,1.152606e+14,...,0.020414,98.625979,0.459104,83.154286,21.078551,4.617874,0.021496,97.338083,0.453109,neutral
5,1000176734,0.007736,159.99,2021-02-08,100.555556,-0.0149,0.0030,True,54.119,3.825599e+13,...,0.007397,61.656543,0.604172,158.398073,78.988624,0.801580,0.007855,63.315723,0.620431,neutral


In [51]:
#tmp_df[['item_code','avg_conv_rate','last_price','PE','PE_err','has_enough_stat','pc2','price_up','price_down','Margin_base','Margin_up','Margin_err_up','Margin_down','Margin_err_down','Group_flag']].head(400)


In [52]:
tmp_df = tmp_df.sort_index()

In [53]:
#I AM HERE NEED TO UPDATE THIS
tmp_df['Margin_rate_to_base'] = np.where(tmp_df['Group_flag'] == 'neutral', 0.,np.where(tmp_df['Group_flag'] == 'neutral_up', np.log(tmp_df['Margin_up']/tmp_df['Margin_base']),np.log(tmp_df['Margin_down']/tmp_df['Margin_base'])))
#tmp_df['Margin_rate_to_last'] = np.log(tmp_df['Margin_down']/tmp_df['Margin_last'])

In [54]:
list_of_columns_down.append('Margin_err_down')

In [55]:
list_of_columns_up.append('Margin_err_up')

In [56]:
tmp_df[list_of_columns_down]



,item_code,avg_conv_rate,avg_unique_views,last_date,last_price,price_down,PE,PE_err,Sales_PE,Sales_PE_err,Delta_price,pc2,avg_sales,avg_sales_down,avg_sales_err_down,Margin_base,Margin_down,Group_flag,Margin_err_down
0,1000022179,0.003121,93.258739,2021-02-06,479.99,475.214020,-0.0136,0.0033,-7.0455,2.1165,126.482941,276.870,0.291093,0.312341,0.006611,36.818246,38.252282,neutral,0.809610
1,1000145006,0.004873,188.190476,2021-02-04,289.99,287.104551,-0.0086,0.0025,-10.1897,2.9552,54.870076,188.819,0.917136,1.015517,0.030011,50.323342,53.259126,neutral,1.573914
2,1000005727,0.017079,170.099034,2021-02-25,99.99,98.995083,-0.0246,0.0044,-2.6853,0.8351,43.296210,40.729,2.905135,2.984204,0.024921,125.781343,126.709720,neutral,1.058153
3,1000016028,0.016290,26.446154,2021-02-25,139.99,137.218012,-0.0328,0.0057,-4.4643,0.8350,65.208655,52.430,0.430796,0.471029,0.007866,28.091607,29.617959,neutral_down,0.494620
4,1000175476,0.055748,80.722222,2021-02-25,83.99,83.154286,-0.0505,0.0061,-2.5839,0.4655,21.780832,48.799,4.500081,4.617874,0.021496,98.015502,97.338083,neutral,0.453109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
705,1000012855,0.003085,70.735185,2021-02-25,449.99,445.512525,-0.0106,0.0028,-7.0734,2.1955,140.413857,237.729,0.218215,0.234209,0.005142,30.640387,32.004964,neutral,0.702669
706,1000194221,0.025424,40.125730,2021-01-08,159.99,NaN,-0.0219,0.0041,-2.5069,0.5744,31.096378,103.349,1.020158,NaN,NaN,31.723225,NaN,neutral_up,NaN
707,1000109909,0.012308,55.833333,2021-02-25,239.99,237.602060,-0.0197,0.0023,-3.0989,0.6622,125.582269,76.090,0.687220,0.708850,0.004694,86.302682,87.596557,neutral,0.580064
708,1000153525,0.008334,73.043478,2021-02-22,104.99,103.945332,-0.0379,0.0085,-2.4168,0.7736,65.687891,22.539,0.608769,0.623661,0.004825,39.988732,40.419458,neutral,0.312685


In [57]:
tmp_df[list_of_columns_up]

,item_code,avg_conv_rate,avg_unique_views,last_date,last_price,price_up,PE,PE_err,Sales_PE,Sales_PE_err,Delta_price,pc2,avg_sales,avg_sales_up,avg_sales_err_up,Margin_base,Margin_up,Group_flag,Margin_err_up
0,1000022179,0.003121,93.258739,2021-02-06,479.99,484.813980,-0.0136,0.0033,-7.0455,2.1165,126.482941,276.870,0.291093,0.271289,0.005742,36.818246,35.413232,neutral,0.749521
1,1000145006,0.004873,188.190476,2021-02-04,289.99,292.904448,-0.0086,0.0025,-10.1897,2.9552,54.870076,188.819,0.917136,0.828287,0.024478,50.323342,47.476716,neutral,1.403032
2,1000005727,0.017079,170.099034,2021-02-25,99.99,100.994916,-0.0246,0.0044,-2.6853,0.8351,43.296210,40.729,2.905135,2.828162,0.023618,125.781343,124.836973,neutral,1.042514
3,1000016028,0.016290,26.446154,2021-02-25,139.99,NaN,-0.0328,0.0057,-4.4643,0.8350,65.208655,52.430,0.430796,NaN,NaN,28.091607,NaN,neutral_down,NaN
4,1000175476,0.055748,80.722222,2021-02-25,83.99,84.834114,-0.0505,0.0061,-2.5839,0.4655,21.780832,48.799,4.500081,4.385293,0.020414,98.015502,98.625979,neutral,0.459104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
705,1000012855,0.003085,70.735185,2021-02-25,449.99,454.512475,-0.0106,0.0028,-7.0734,2.1955,140.413857,237.729,0.218215,0.203313,0.004464,30.640387,29.320616,neutral,0.643734
706,1000194221,0.025424,40.125730,2021-01-08,159.99,163.222012,-0.0219,0.0041,-2.5069,0.5744,31.096378,103.349,1.020158,0.970271,0.011146,31.723225,32.807133,neutral_up,0.376888
707,1000109909,0.012308,55.833333,2021-02-25,239.99,242.401940,-0.0197,0.0023,-3.0989,0.6622,125.582269,76.090,0.687220,0.666251,0.004412,86.302682,85.019646,neutral,0.563000
708,1000153525,0.008334,73.043478,2021-02-22,104.99,106.045167,-0.0379,0.0085,-2.4168,0.7736,65.687891,22.539,0.608769,0.594232,0.004597,39.988732,39.560772,neutral,0.306042


In [ ]:
#print(tmp_df['Margin_rate_to_base'].mean()*100)

In [58]:
tmp_df.to_csv('../raw_data/Excel_files/Margin_result_Sales_PE.csv')

In [59]:
tmp_df_only_change=tmp_df[tmp_df['Group_flag']!='neutral']


In [60]:
tmp_df_only_change['Margin_rate_to_base'].mean()

0.13756086777524523

In [61]:
tmp_df_only_change['Margin_rate_to_base'].nlargest(10)


198    0.689917
672    0.367043
8      0.333292
544    0.330028
367    0.311246
437    0.298487
426    0.296314
520    0.290302
123    0.290195
184    0.285127
Name: Margin_rate_to_base, dtype: float64

In [68]:
tmp_df_only_change['Margin_rate_to_base'].median()


0.1045135592381947